In [4]:
from Common import *

In [5]:
nlp = spacy.load('en_core_web_lg')

In [6]:
??Render

In [7]:
# add the text classifier to the pipeline if it doesn't exist
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'textcat' not in nlp.pipe_names:
    textcat = nlp.create_pipe('textcat')
    nlp.add_pipe(textcat, last=True)
# otherwise, get it, so we can add labels to it
else:
    textcat = nlp.get_pipe('textcat')

In [8]:
# add label to text classifier
textcat.add_label('POSITIVE')

1

In [30]:
import thinc.extra.datasets
import random
from pathlib import Path
from spacy.util import minibatch, compounding

In [14]:
def load_data(limit=0, split=0.8):
    """Load data from the IMDB dataset."""
    # Partition off part of the train data for evaluation
    train_data, _ = thinc.extra.datasets.imdb()
    random.shuffle(train_data)
    train_data = train_data[-limit:]
    texts, labels = zip(*train_data)
    cats = [{'POSITIVE': bool(y)} for y in labels]
    split = int(len(train_data) * split)
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])


In [16]:
#"Model name. Defaults to blank 'en' model."
model = 'en_core_web_lg'

output_dir = 'data\\textcategorizer'

#"Number of texts to train from"
n_texts = 1000 

#"Number of training iterations"
n_iter = 20

In [19]:
print("Loading IMDB data...")
(train_texts, train_cats), (dev_texts, dev_cats) = load_data(limit=n_texts)



Loading IMDB data...


In [21]:
train_texts[:3]

('Most action films are crass of Hindi cinema, especially of Sunny and his family \n\n\n\nThe film is typical Sunny type with bashes, big dialogues and melodrama\n\n\n\nThe film also has typical Rajiv Rai ingredients of many henchmen and a weird villain\n\n\n\nThe starting is okay and then the shift to Kenya is good but then the film goes on and on \n\n\n\nThe sequence of events move at a slow pace and nothing that great happens\n\n\n\nThey are many stupid scenes like the Kenya policemen are shown like jokers especially Sharat\n\n\n\nThe climax too is prolonged\n\n\n\nRajiv Rai does an okay job Music is okay, only 1 song works and that is the last TOOFAN Camera-work is good\n\n\n\nSunny Deol is as usual, Chunky acts like a monkey while his serious scenes are laughable, Naseer is alright heroines are pure wood Amrish Puri is not even half as scary as he was in TRIDEV the rest are okay',
 "It's a rather good movie, but too Americanised in it's predictability. Change the Kung Fu for footb

In [22]:
train_cats[:3]

[{'POSITIVE': False}, {'POSITIVE': True}, {'POSITIVE': False}]

In [23]:
dev_texts[:3]

("Are we really making 'video nasties' again? In the guise of a digital wide screen big budget remake of 8MM, this is quite a ride. Unfortunately there is a bit too much story and at times this becomes like a travelogue as our heroine searches the sleaze spots of Paris, Hamburg and Amsterdam. I am however being rather churlish for the 'depraved' scenes, including everything from, hot wax, harsh whipping and rough sex to drowning, beheading and some. These scenes are immaculate and it's a pity Bruno and his budget couldn't stretch to make all the many characterful creatures introduced become more than simply caricatures.",
 "This movie was advertised on radio, television, magazines, etc. Almost every hour or every issue. So when we went to the Kinnepolis multiplex our expectations were very high. But oh boy, how sad this movie is! It is a movie in Hollywood style about a movie in a movie. Shades shows so clear we aren't ready to produce 'big Hollywood movies'. I am not a movie critic, b

In [24]:
dev_cats[:3]

[{'POSITIVE': True}, {'POSITIVE': False}, {'POSITIVE': False}]

In [25]:
print("Using {} examples ({} training, {} evaluation)".format(n_texts, len(train_texts), len(dev_texts)))
train_data = list(zip(train_texts,  [{'cats': cats} for cats in train_cats]))


Using 1000 examples (800 training, 200 evaluation)


In [26]:
train_data[:3]

[('Most action films are crass of Hindi cinema, especially of Sunny and his family \n\n\n\nThe film is typical Sunny type with bashes, big dialogues and melodrama\n\n\n\nThe film also has typical Rajiv Rai ingredients of many henchmen and a weird villain\n\n\n\nThe starting is okay and then the shift to Kenya is good but then the film goes on and on \n\n\n\nThe sequence of events move at a slow pace and nothing that great happens\n\n\n\nThey are many stupid scenes like the Kenya policemen are shown like jokers especially Sharat\n\n\n\nThe climax too is prolonged\n\n\n\nRajiv Rai does an okay job Music is okay, only 1 song works and that is the last TOOFAN Camera-work is good\n\n\n\nSunny Deol is as usual, Chunky acts like a monkey while his serious scenes are laughable, Naseer is alright heroines are pure wood Amrish Puri is not even half as scary as he was in TRIDEV the rest are okay',
  {'cats': {'POSITIVE': False}}),
 ("It's a rather good movie, but too Americanised in it's predicta

In [28]:
textcat.name, textcat.labels

('textcat', ['POSITIVE'])

In [32]:
def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 1e-8  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 1e-8  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_score = 2 * (precision * recall) / (precision + recall)
    return {'textcat_p': precision, 'textcat_r': recall, 'textcat_f': f_score}


In [93]:
n_iter = 2

In [100]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()
    print("Training the model...")
    print('{:^5}\t{:^5}\t{:^5}\t{:^5}'.format('LOSS', 'precision', 'recall', 'f_score'))
    for i in range(n_iter):
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)
        with textcat.model.use_params(optimizer.averages):
            # evaluate on the dev data split off in load_data()
            scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print('{0:.3f}\t{1:>9.3f}\t{2:>6.3f}\t{3:>7.3f}'  # print a simple table
              .format(losses['textcat'], scores['textcat_p'],
                      scores['textcat_r'], scores['textcat_f']))

Training the model...
LOSS 	precision	recall	f_score
3.737	    0.737	 0.745	  0.741
1.804	    0.703	 0.796	  0.746


In [54]:
# test the trained model
test_text = """ It's a rather bad movie.  """
doc = nlp(test_text)
print(test_text, doc.cats)

 It's a rather bad movie.   {'POSITIVE': 0.3549625277519226}


In [102]:
doc.mem.size

3360

In [45]:
train_texts[1], train_cats[1]

("It's a rather good movie, but too Americanised in it's predictability. Change the Kung Fu for football and the Turkish Family for a Pakistani one, and you get to watch Bend It Like Beckham (2002) almost scene for scene. A nice feature the serves as the backbone of the movie is the progression of fights with the mysterious ninja under the highway, beginning with miserable losses and slowly progressing until the last fight is a win against oneself, as the Kung Fu master stressed several times. On a different level, the Danish life is revealed quite different than the image it has by outsiders: the non indigenous immigrants that make a large proportion (actually, the majority) of the Danish citizenry, the graffiti in the Copenhagen suburbs, the taunting of the immigrant girl in the begging of the movie. All portray a different picture than one has in mind when one hears the word Denmark.",
 {'POSITIVE': True})

In [71]:
batches = minibatch(train_data, size=compounding(4., 32., 1.001))

In [69]:
len(train_data)
train_data[0]

('Most action films are crass of Hindi cinema, especially of Sunny and his family \n\n\n\nThe film is typical Sunny type with bashes, big dialogues and melodrama\n\n\n\nThe film also has typical Rajiv Rai ingredients of many henchmen and a weird villain\n\n\n\nThe starting is okay and then the shift to Kenya is good but then the film goes on and on \n\n\n\nThe sequence of events move at a slow pace and nothing that great happens\n\n\n\nThey are many stupid scenes like the Kenya policemen are shown like jokers especially Sharat\n\n\n\nThe climax too is prolonged\n\n\n\nRajiv Rai does an okay job Music is okay, only 1 song works and that is the last TOOFAN Camera-work is good\n\n\n\nSunny Deol is as usual, Chunky acts like a monkey while his serious scenes are laughable, Naseer is alright heroines are pure wood Amrish Puri is not even half as scary as he was in TRIDEV the rest are okay',
 {'cats': {'POSITIVE': False}})

In [72]:
for batch in batches:
    print(batch[0])

('Most action films are crass of Hindi cinema, especially of Sunny and his family \n\n\n\nThe film is typical Sunny type with bashes, big dialogues and melodrama\n\n\n\nThe film also has typical Rajiv Rai ingredients of many henchmen and a weird villain\n\n\n\nThe starting is okay and then the shift to Kenya is good but then the film goes on and on \n\n\n\nThe sequence of events move at a slow pace and nothing that great happens\n\n\n\nThey are many stupid scenes like the Kenya policemen are shown like jokers especially Sharat\n\n\n\nThe climax too is prolonged\n\n\n\nRajiv Rai does an okay job Music is okay, only 1 song works and that is the last TOOFAN Camera-work is good\n\n\n\nSunny Deol is as usual, Chunky acts like a monkey while his serious scenes are laughable, Naseer is alright heroines are pure wood Amrish Puri is not even half as scary as he was in TRIDEV the rest are okay', {'cats': {'POSITIVE': False}})
('And when I watch Sarah Silverman, I get the same results. I love qui

('Nobody truly understands the logic behind the numbering of Italian zombie-flicks, but \x96 honestly \x96 why would we bother? Every single film in the Zombi-"series" delivers great fun, nasty gore and gratuitous shocks and "Zombi 3" is no exception to this, despite all the production difficulties that occurred whilst shooting. This film began as an interesting Lucio Fulci project, who had to elaborate further on his "Zombi 2" success, but it ended up being a typical Bruno Mattei product with more flaws and stolen ideas from previous films. The screenplay is hopelessly inept and ignores all forms of continuity, every ingenious idea from George A. Romero\'s "Night of the Living Dead" and "The Crazies" is shamelessly repeated here and the acting performances are truly miserable and painful to look at. Yet all this didn\'t upset me for one moment because the sublime over-the-top gore compensates for everything! On a secret army base at the Phillipines, scientists completed the bacterial 

('i can\'t believe that NONE of the official reviews for this movie warn people that it contains two quite upsetting sexual assault scenes. It\'s as though our culture accepts this kind of behavior as simply sexual but not violent. My biggest problem with the movie is that it doesn\'t seem to condemn these assaults - as in, the woman who is repeatedly assaulted and pressured never holds the men accountable for their actions, and neither does anyone else. One man is stopped from completing the assault when someone throws a dagger at him, but he is reprimanded only with "you cannot force a woman to love you" rather than "you should never force a woman sexually, you jerk"... From a woman\'s point of view, the movie is a let down. It sort of "throws a bone" to women in letting them be both skilled fighters and leaders, but the movie is much more defined by the romance - which is characterized by the notion that human sexuality must involve an imbalance of power, with men dominating the wom

('This delectable fusion of New Age babble and luridly bad film-making may not "open" you up, to borrow one of the film\'s favorite verbs, but it might leave your jaw slack and your belly sore from laughter or retching. Based on the best-selling book by James Redfield, first (self) published in 1993, this cornucopia of kitsch tracks the spiritual awakening of an American history teacher (Matthew Settle) who, on traveling to deepest, darkest, phoniest Peru and sniffing either the air or something else more illegal. Namely what he discovers is a schlock Shangri La populated by smiling zombies who may be nuts or just heavily medicated, perhaps because they\'re often accompanied by a panpipe flourish and an occasional shout out from a celestial choir. Although there\'s a lot of talk about "energy," that quality is decidedly missing from the motley cast whose numbers include Thomas Kretschmann, Annabeth Gish, Hector Elizondo and Jurgen Prochnow, all of whom are now firmly ensconced in the c

('This is an excellent modern-day film noir...."excellent" in that it\'s interesting, start-to-finish. There are some holes in here and some goofy parts that make you shake your head in disbelief.....but I haven\'t found anyone who didn\'t get caught up in this story. The movie has the right amount of action, suspense, plot twists and interesting characters. In addition, it sports some nice colors and cinematography plus a good guitar-based soundtrack. \n\n\n\nI labeled this crime movie a "film noir" because it\'s gritty and the all the characters are no good. Even the only supposed-good guy, played by Nicholas Cage, gets himself in trouble by lying and has a quick affair he should\'t have. He also does something at the end which isn\'t right, but I\'m not going the spoil it by saying. Suffice to say, however, that the rest of the characters are so bad they make Cage look good!\n\n\n\nSpeaking of "bad guys," does anyone do it better than Dennis Hopper? Not many. At least in the "derang

...

('"La Furia del Hombre Lobo" forms a completely stand-alone storyline which doesn\'t seem to fit in at all with the previous Waldemar Daninsky movies. Some have commented that this movie is supposed to take place before the events of "Werewolf Shadow", although it was released afterwards ... they may be right, I\'m not sure. Anyway, in this movie Waldemar Daninsky is bitten by a yeti-like creature in Tibet (great dialogue here -- "It was a yeti. But that\'s impossible. I\'m a scientist and these things don\'t exist. It was a hallucination. That\'s all.") and although marked with the sign of the pentagram, he is able to prevent the change into a werewolf until he discovers that his wife has been cheating on him. Changing into the beast one night, he kills both her and her lover before running out into a storm and being electrocuted. It\'s not long before he\'s resurrected by a dominatrix university professor who is conducting some kind of unfathomable experiments with mind control. He i

('No scenario, bad actors (poor Melissa Gilbert)... Beurk Beurk Beurk ...\n\n\n\nGive a such budget to make this... In Belgium, we make ten films which win all prices in Cannes with this.\n\n\n\nLast time that I\'ve seen a such NULL-Film was Hypercube. But scenario was better.\n\n\n\nIs anyone knows if the director was a graduate in school-film or a cop ?\n\n\n\nThe better things in this film was the word "End".\n\n\n\nWhy authorize to sell this ? 1ç is to expensive. \n\n\n\nI\'ve pay ten dollars to buy this.\n\n\n\nFor me, pay for this was my BIG MISTAKE of millennium.\n\n\n\nToo bad.\n\n\n\nNext time I\'ll break my arm but buy this type of sh*t.', {'cats': {'POSITIVE': False}})
('"Johnny Dangerously" is a sort of hit and miss comedy that has it\'s laughs and "huh?". But I suggest to give it a chance. I think it is greatly over looked. Not too many people give this movie a chance. It does work. Just think of it as a little parody of "Goodfellas". Michael Keaton is very funny in his ro

('A sentimental school drama set in Denmark, 1969, "We Shall Overcome" offers a pathetic Danish take on US culture. Frits (Janus Dissing Rathke), a flower-power obsessed, naive 13-year-old, exits with half his ear hanging off from brutal master Lindum-Svendsen\'s (Bent Mejding) office. Lindum-Svendsen, a school director, portrayed as a fascistoid tyrant, has the local community in control. Lindum-Svendsen\'s gone too far this time, and with his father, recovering from a mental breakdown (sure, there wasn\'t enough drama already..), and overly stereotyped hippie music teacher Mr Svale (\'Hi, call me Freddie\'), Frits stands up for justice.\n\n\n\nTell you what. It\'s so unconvincing, over-(method-)acted, and so full of misery, that as a \'family\' picture this grotesque -filled with cliché\'s- excuse for a movie fails miserably to convince non-Scandinavian audiences. Sorry, kind danish readers, to crash like this into your sentimental journeys.. But it\'s definitely NOT a tale about a \

('i love bad shark movies. i really do. i laugh hysterically at them. and the scifi channel was having a marathon of them, culminating in the premier of their new original picture, hammerhead: shark frenzy. based on the previews, it looked like it was going to be HIGHLY amusing. essentially a remake of benchley\'s creature, really. it was prefaced by a showing of shark attack 3: megalodon, which is shark movie hilarity at its best. i was in the mood; i was ready to go. bring it on, hammerhead-mad-scientist-man! oh, god, was that movie wrong.\n\n\n\nwrong, wrong, wrong.\n\n\n\nsick. twisted. MESSED UP.\n\n\n\nthis is theoretical reproduction at its very worst, my friends. when a drugged-out girl is brought out of suspended animation and strapped to a table screaming her head off because the shark-human hybrid fetus the absolutely insane "scientist" deliberately implanted in her womb wants OUT... Jesus monkeys. that\'s what i call disturbing.\n\n\n\nthat\'s really how the plot works: hmm

('Well this movie actually made me feel so strongly that I signed up for an IMDb account just to warn people. It is patently AWFUL!! NOTHING makes sense in this movie. There is way too many subplots for a start. Josh Hartnett\'s character is an aspiring actor and yoga instructor as well as a cop who seems to be living way beyond his means and only teaches yoga to hot girls (Some of whom wait naked in his jacuzzi for when he comes back from work). Add to that the fact that his dad was killed by a crooked cop who just so happens to be in on the current crime being investigated by the hapless duo. Harrison Ford\'s character is trying to sell real estate on the side and is sleeping with the Internal Affairs investigator\'s ex-wife who happens to run a psychic radio show which Ford\'s character calls from time to time. NONSENSE!!\n\n\n\nI can\'t remember the characters names (that\'s how forgettable this is) so I\'ll refer to them as Ford and Hartnett.\n\n\n\nThen there\'s the dialogue whic

In [85]:
output_dir = 'data\\textcategorizer'

In [88]:
win_dir = os.path.join(os.path.abspath('.'), output_dir)
if not os.path.exists(win_dir):
    os.makedirs(win_dir)
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to I:\ML\courses\NLP\data\textcategorizer


In [84]:

win_dir = os.path.join(os.path.abspath('.'), output_dir)
Path(win_dir)

WindowsPath('I:/ML/courses/NLP/data/textcategorizer')

In [78]:
import os

False

In [91]:
??optimizer.to_gpu